In [11]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split,  cross_val_score, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, recall_score, ConfusionMatrixDisplay
from mlxtend.plotting import plot_decision_regions
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
from typing import Tuple

DIGITS=3

In [12]:
def load_dataset() -> np.ndarray:
    dataset = load_iris()

    return dataset


def split_dataset(dataset) -> np.ndarray:
    X_train, X_test, y_train, y_test = train_test_split(dataset['data'], dataset['target'], test_size = 0.2)
    
    return X_train, X_test, y_train, y_test

# region X
# def split_data(feature_data: np.ndarray, target_data: np.ndarray, test_data_rate: float) ->  np.ndarray:
#     if feature_data is None:
#         raise Exception('Error : no input feature data')

#     if test_data_rate is None:
#         raise Exception('Error : no input test data rate')

#     if test_data_rate >= 1 or test_data_rate <= 0:
#         raise Exception('Error : test data rate must be a float value between 0 and 1')

#     train_data, test_data, train_target, test_target = train_test_split(feature_data, target_data,   
#     test_size=test_data_rate, shuffle=True)

#     return train_data, test_data, train_target, test_target
# endregion

In [13]:
def get_features_name(dataset) -> np.ndarray:
    return dataset.feature_names

def get_targets_name(dataset) -> np.ndarray:
    return dataset.target_names

def get_min_max_feature_data(data:np.ndarray) -> np.ndarray:
    min_values = np.min(data, axis=0)
    max_values = np.max(data, axis=0)

    return min_values, max_values
    
def print_min_max_feature_data(data: np.ndarray, features: np.ndarray):
    number_of_feature = data.shape[1]    

    for i in range(0, number_of_feature):
        min_value = round(min(data[:, i]), DIGITS)
        max_value = round(max(data[:, i]), DIGITS)

        print("feature(" + features[i] + ")'s range: " + str(min_value) + ' to ' + str(max_value)) 

def visualizing_dataset(dataset):
    col_names = list(dataset.feature_names)
    col_names.append('target')

    df = pd.DataFrame(np.c_[dataset.data, dataset.target], columns=col_names)
    display(df)

    sns.pairplot(df, hue='target', vars=dataset.feature_names)    
    
    # plt.figure(figsize=(20,10))
    # sns.heatmap(df.corr(), annot=True,square=True,cmap='RdBu')
    # annot=True 데이터 값 셀 안에 적어줌, square=True 정사각형으로 출력    
    # Color Brewer 팔레트 중 diverging : Diverging 팔레트는 양쪽으로 강조가 되기 때문에 낮은 값과 높은 값에 모두 관심을 가져야 하는 데이터 세트에 적합

In [14]:
def set_scaler_train(train_data: np.ndarray) ->  np.ndarray:
    scaler = StandardScaler()
    
    train_std = scaler.fit_transform(train_data)
    # print_min_max_feature_data(X_train_std)

    return train_std

def set_scaler_train_test(train_data: np.ndarray, test_data: np.ndarray) ->  np.ndarray:
    scaler = StandardScaler()

    # scaler.fit(X_train)
    train_std = scaler.fit_transform(train_data)
    test_data_std = scaler.transform(test_data)

    # print_min_max_feature_data(X_train_std)
    return train_std, test_data_std

def set_scaler_train_validation_test(train_data: np.ndarray, validation_data: np.ndarray, test_data: np.ndarray) ->  np.ndarray:
    scaler = StandardScaler()

    # scaler.fit(X_train)
    train_std = scaler.fit_transform(train_data)
    validation_std = scaler.transform(validation_data)    
    test_data_std = scaler.transform(test_data)

    # print_min_max_feature_data(X_train_std)
    return train_std, validation_std, test_data_std

In [15]:
# Set classification report parameter value
# TARGET_NAMES=['class 0', 'class 1', 'class 2']
# TARGET_NAMES=get_targets_name()

def get_confusion_matric(test_target: np.ndarray, predicted_target: np.ndarray) -> np.ndarray:
    return confusion_matrix(test_target, predicted_target)

def cal_confusion_matric(test_target: np.ndarray, predicted_target: np.ndarray, targets_name: np.ndarray) -> np.ndarray:  
    matric = confusion_matrix(test_target, predicted_target) # , labels=[0, 1, 2]
    
    report = classification_report(test_target, predicted_target, target_names=targets_name, digits=DIGITS)    

    ACC = accuracy_score(test_target, predicted_target)
    
    FP = matric.sum(axis=0) - np.diag(matric) # 세로에서 TP 뺴기    
    FN = matric.sum(axis=1) - np.diag(matric) # 가로에서 TP 빼기    
    TP = np.diag(matric)    
    TN = matric.sum() - (FP + FN + TP)
    
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN) # TPR = recall_score(y_test, predicted_target, average=None or 'macro)    
     # False negative rate
    FNR = FN/(TP+FN) # FNR = 1 - TPR    
    # Specificity or true negative rate
    TNR = TN/(TN+FP)     
    # Fall out or false positive rate
    FPR = FP/(FP+TN) # FPR = 1 - TNR
    
    # region X
    # Precision or positive predictive value
    # PPV = TP/(TP+FP)
    # Negative predictive value
    # NPV = TN/(TN+FN)
    # False discovery rate
    # FDR = FP/(TP+FP)
    # # Overall accuracy
    # ACC = (TP+TN)/(TP+FP+FN+TN) 

    # print("TPR :", TPR)
    # print("FNR :", FNR)
    # print("TNR :", TNR)
    # print("FPR :", FPR)
    # endregion

    return matric, report, ACC, TPR, FNR, TNR, FPR

In [16]:
# Set classifier parameter value
C=10
MAX_ITER=10000
GAMMA='auto' # rbf, poly, sigmoid
DEGREE=3 # poly

def set_svm_model(kernel: str):
    if kernel == "linear":
        svm_model = svm.SVC(kernel="linear", C=C, max_iter=MAX_ITER)
    elif kernel == "rbf":
        svm_model = svm.SVC(kernel='rbf', C=C, gamma=GAMMA, max_iter=MAX_ITER)
    elif kernel == "poly":
        svm_model = svm.SVC(kernel='poly', C=C, gamma=GAMMA, degree=DEGREE, max_iter=MAX_ITER)  
    elif kernel == "sigmoid":
        svm_model = svm.SVC(kernel='sigmoid', C=C, gamma=GAMMA, max_iter=MAX_ITER)  
    elif kernel == "LinearSVC":
        svm_model = svm.LinearSVC(C=C, max_iter=MAX_ITER)
    
    return svm_model

def set_fit_svm_model(kernel: str, train_data_std: np.ndarray, train_target: np.ndarray):
    svm_model = set_svm_model(kernel)
    
    return svm_model.fit(train_data_std, train_target)

def predict_svm_classification(svm_model, test_data_std: np.ndarray, test_target: np.ndarray, targets_name: np.ndarray, now_date_time):        
    predicted_target = svm_model.predict(test_data_std)
    # print("정답 라벨:", test_target)
    # print("예측 라벨:", predicted_target)   
        
    cfm, report, acc, tpr, fnr, tnr, fpr = cal_confusion_matric(test_target, predicted_target, targets_name)

    acc = np.round(acc, DIGITS)
    tpr = np.round(tpr, DIGITS)
    fnr = np.round(fnr, DIGITS)
    tnr = np.round(tnr, DIGITS)
    fpr = np.round(fpr, DIGITS)
    
    if hasattr(svm_model, 'kernel') == True:
        print("\nkernel(" + svm_model.kernel + ")'s Confusion metric:")
        cfm = get_confusion_matric(test_target, predicted_target)        
        disp_cfm = ConfusionMatrixDisplay.from_estimator(svm_model, test_data_std, test_target, display_labels=targets_name, cmap=plt.cm.Blues, normalize=None)
        disp_cfm.ax_.set_title("Confusion matrix")
        # plt.figure()        
        plt.savefig('results/' + 'Confusion matrix (' + svm_model.kernel + ') ' + now_date_time + '.jpg')
        plt.show()

        print("kernel(" + svm_model.kernel + ")'s Classification report:\n", report)
        print("kernel(" + svm_model.kernel + ")'s Accuracy:", acc)    
        # print("kernel(" + kernel + ")'s prediction accuracy: {:.2f}".format(np.mean(predicted_target == test_target)))    
        # print("kernel(" + svm_model.kernel + ")'s TPR: ", tpr)
        # print("kernel(" + svm_model.kernel + ")'s FNR: ", fnr)
        # print("kernel(" + svm_model.kernel + ")'s TNR: ", tnr)
        # print("kernel(" + svm_model.kernel + ")'s FPR: ", fpr)   
    else:
        print("\nLinearSVC's Confusion metric:")
        cfm = get_confusion_matric(test_target, predicted_target)
        disp_cfm = ConfusionMatrixDisplay.from_estimator(svm_model, test_data_std, test_target, display_labels=targets_name, cmap=plt.cm.Blues, normalize=None)
        disp_cfm.ax_.set_title("Confusion matrix")
        # plt.figure()        
        plt.savefig('results/Confusion matrix (LinearSVC) ' + now_date_time + '.jpg')
        plt.show()

        print("LinearSVC's Classification report:\n", report)
        print("LinearSVC's Accuracy:", acc) 
        # print("LinearSVC's TPR:", tpr)
        # print("LinearSVC's FNR:", fnr)
        # print("LinearSVC's TNR:", tnr)
        # print("LinearSVC's FPR:", fpr)    
   
    return acc, tpr, fnr, tnr, fpr

In [17]:
# Cross-validation 
def cross_validation_svm(kernel: str, cv: int, train_test_data: np.ndarray, train_test_target: np.ndarray, standardscaler: bool):
    if standardscaler == True:
        scaler = StandardScaler()
        svm_model = set_svm_model(kernel)
        pipeline = Pipeline([('scaler', scaler), ('model', svm_model)])

        scores = cross_val_score(pipeline, train_test_data, train_test_target, cv=cv)
    else:
        svm_model = set_svm_model(kernel)

        scores = cross_val_score(svm_model, train_test_data, train_test_target, cv=cv) # scoring='accuracy'
    
    if hasattr(svm_model, 'kernel') == True:
        print("\nkernel(" + kernel + ")'s Cross-validation report:\n", pd.DataFrame(cross_validate(svm_model, train_test_data, train_test_target, cv=cv)))
        print("kernel(" + kernel + ")'s Cross-validation Accuracy:", round(scores.mean(), DIGITS))
    
    else:
        print("\nLinearSVC's Cross-validation report:\n", pd.DataFrame(cross_validate(svm_model, train_test_data, train_test_target, cv=cv)))
        print("LinearSVC's Cross-validation Accuracy:", round(scores.mean(), DIGITS))       

In [18]:
# list_nu = np.arange(0.001, 1.001, 0.001)
# for nu in range(0, len(list_nu)):
#     nu_value = list_nu[nu]
#     gamma_value = 'auto'

# parameters = {'C': list_nu}
# PARAMETERS = {'C': [0.001, 0.01, 0.1, 1, 10, 25, 50, 100]}
# PARAMETERS = {'C': [0.001, 0.01, 0.1, 1, 10, 25, 50, 100],\
#               'gamma':[0.001, 0.01, 0.1, 1, 10, 25, 50, 100]}

# GridSearchCV
def gridsearchCV_svm(kernel: str, cv: int, params: dict, train_test_data_std: np.ndarray, train_test_target: np.ndarray, standardscaler: bool):
    if standardscaler == True:
        scaler = StandardScaler()
        svm_model = set_svm_model(kernel)
        pipeline = Pipeline([('scaler', scaler), ('model', svm_model)])

        grid_svm_model = GridSearchCV(pipeline, param_grid=params, cv=cv, verbose=1)
        grid_svm_model.fit(train_test_data_std, train_test_target)

    else:
        svm_model = set_svm_model(kernel)
    
        grid_svm_model = GridSearchCV(svm_model, param_grid=params, cv=cv, verbose=1)
        grid_svm_model.fit(train_test_data_std, train_test_target)
    
    result = pd.DataFrame(grid_svm_model.cv_results_['params'])
    result['Accuracy'] = grid_svm_model.cv_results_['mean_test_score']
    # result['mean_test_score'] = grid_svm_model.cv_results_['mean_test_score']    

    if hasattr(svm_model, 'kernel') == True:
        print("\nkernel(" + kernel + ")'s GridsearchCV Accuracy:\n", result.sort_values(by='Accuracy', ascending=False))
        print("kernel(" + kernel + ")'s GridsearchCV Best params:", grid_svm_model.best_params_)

    else:
        print("\nLinearSVC's GridsearchCV Accuracy:\n", result.sort_values(by='Accuracy', ascending=False))  
        print("LinearSVC's GridsearchCV Best params:", grid_svm_model.best_params_)

In [19]:
# Visualizing SVM
def visualizing_svm_prediction(kernel: str, features_name: list, targets_name: list, x_train_std: np.ndarray, y_train: np.ndarray):
    plot_unit = 0.025
    
    for i in range(0, len(features_name), 2):
        selected_train_std = x_train_std[:, i:i+2] # select ith and i+1th features        
        model = set_fit_svm_model(kernel, selected_train_std, y_train)

        feature_min_values, feature_max_values = get_min_max_feature_data(selected_train_std)

        fir_feature_min, fir_feature_max = feature_min_values[0]-1, feature_max_values[0]+1
        sec_feature_min, sec_feature_max = feature_min_values[1]-1, feature_max_values[1]+1

        fir_feature, sec_feature = np.meshgrid(np.arange(fir_feature_min, fir_feature_max, plot_unit), np.arange(sec_feature_min, sec_feature_max, plot_unit))

        predicted_target = model.predict(np.c_[fir_feature.ravel(), sec_feature.ravel()])
        predicted_target = predicted_target.reshape(fir_feature.shape)

        fir_target = selected_train_std[y_train==0] 
        sec_target = selected_train_std[y_train==1]
        thd_target = selected_train_std[y_train==2]  
                
        plt.pcolormesh(fir_feature, sec_feature, predicted_target, alpha=0.1)        
        # plt.scatter(selected_train_std[:, 0], selected_train_std[:, 1], c=y_train)        
        plt.scatter(fir_target[:, 0], fir_target[:, 1])
        plt.scatter(sec_target[:, 0], sec_target[:, 1])
        plt.scatter(thd_target[:, 0], thd_target[:, 1])
        plt.xlabel(features_name[i])
        plt.ylabel(features_name[i+1])
        plt.legend(targets_name, loc=4)
        plt.xlim(fir_feature.min(), fir_feature.max())        
        # plt.title('Support Vector Machine')
        plt.show()    

        # plt.pcolormesh(fir_feature, sec_feature, predicted_target, alpha=0.1)
        # plt.scatter(selected_train_std[:, 0], selected_train_std[:, 1], c=y_train)        
        # plt.xlabel(features_name[i])
        # plt.ylabel(features_name[i+1])
        # plt.legend(targets_name, loc=4)
        # plt.xlim(fir_feature.min(), fir_feature.max())        
        # # plt.title('Support Vector Machine')
        # plt.show()   
        
def visualizing_svm_decision_region(kernel: str, features_name: list, targets_name:list, x_train_std:np.ndarray, y_train:np.ndarray):
    for i in range(0, len(features_name), 2):
        selected_train_std = x_train_std[:, i:i+2] # select ith and i+1th features        
        model = set_fit_svm_model(kernel, selected_train_std, y_train)
    
        fir_target = selected_train_std[y_train==0] 
        sec_target = selected_train_std[y_train==1]
        thd_target = selected_train_std[y_train==2]        

        # plt.scatter(red_pt[:,0], red_pt[:,1], color='r')
        plt.scatter(fir_target[:,0], fir_target[:,1])
        plt.scatter(sec_target[:,0], sec_target[:,1])
        plt.scatter(thd_target[:,0], thd_target[:,1])
        plt.xlabel(features_name[i])
        plt.ylabel(features_name[i+1])
        plt.legend(targets_name, loc=4) 
        plt.show() 

In [20]:
# dataset = load_dataset()

# X_train, X_test, y_train, y_test = split_dataset(dataset)
# X_train_std, X_test_std = set_scaler(X_train, X_test)

# model = set_fit_svm_model("linear", X_train_std, y_train)